# Part 1 
#### (keep scrolling for part 3)

In [6]:
%pip install lxml #installed lxml package to help parse html files
%pip install wikipedia #installed wikipedia package to help obtain wikipedia page
import numpy as np 
import pandas as pd
import lxml.html #parses html objects
import wikipedia as wp #installed to easily access Wikipedia content
from bs4 import BeautifulSoup #installed to pull data out of HTML and XML files
print('Libraries imported.')

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Libraries imported.


#### obtaining the content from wikipedia using the wp.page code to retreive the page data. the .html and .encode("UTF-8") used to change coding of languge to UTF-8

In [62]:
url=wp.page("List_of_postal_codes_of_Canada:_M").html().encode("UTF-8")
df=pd.read_html(url,header=0)[0]

#### if a borough is classified as 'not assigned', the row will be removed

In [63]:
df = df[df.Borough != 'Not assigned']

#### grouping the data by postal code and borough and joining rows together which may share neighborhoods. Finally reseting the index

In [64]:
df = df.groupby(['Postal Code', 'Borough'])['Neighbourhood'].apply(list).apply(lambda x:', '.join(x)).to_frame().reset_index()

#### if neighborhood value is 'not assigned' the borough value will be assigned in place of the neighborhood value.

In [65]:
for index, row in df.iterrows():
    if row['Neighbourhood'] == 'Not assigned':
        row['Neighbourhood'] = row['Borough']

# Part 2
#### (keep scrolling for part 3)
#### obtaining geospatial data to get the longitude and latitude of the different postal codes

In [50]:
import requests
import io
geo_url="http://cocl.us/Geospatial_data"
Geo_Data=requests.get(geo_url).content
Long_Lat=pd.read_csv(io.StringIO(Geo_Data.decode('utf-8')))

#### Merging the two dataframes

In [66]:
df=pd.merge(Long_Lat,df,on='Postal Code')

#### reordering the columns

In [114]:
Tor_df=df[['Postal Code','Borough','Neighbourhood','Latitude','Longitude']]

# Part 3

In [16]:
%pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
print('Libraries imported.')

     |████████████████████████████████| 112kB 5.3MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.
Libraries imported.


In [68]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="Toronto")
location = geolocator.geocode(address)
latitude_Tor = location.latitude
longitude_Tor = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude_Tor, longitude_Tor))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [69]:
# create map of Toronto using latitude and longitude values
map_Tor = folium.Map(location=[latitude_Tor, longitude_Tor], zoom_start=10)

# add markers to map
for lat, lng, borough, Neighbourhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(Neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Tor)
    
map_Tor

In [108]:
Neighbourhood = pd.DataFrame(columns={'Borough','Neighbourhood','Latitude','Longitude'})
for data in df:
    borough_name = Tor_df['Borough']
    neighbourhood_name = Tor_df['Neighbourhood']
    neighbourhood_lat = Tor_df['Latitude'][0]
    neighbourhood_lon = Tor_df['Longitude'][0]
    Neighbourhood = Neighbourhood.append({'Borough': borough,
                                          'Neighbourhood': neighbourhood_name,
                                          'Latitude': neighbourhood_lat,
                                          'Longitude': neighbourhood_lon},ignore_index=True)
Neighbourhood=pd.DataFrame(Neighbourhood,columns=['Borough','Neighbourhood','Latitude','Longitude'])

,Borough,Neighbourhood,Latitude,Longitude
0,0 Scarborough 1 Scarborough 2 S...,0 Malv...,43.806686,-79.194353
1,0 Scarborough 1 Scarborough 2 S...,0 Malv...,43.806686,-79.194353
2,0 Scarborough 1 Scarborough 2 S...,0 Malv...,43.806686,-79.194353
3,0 Scarborough 1 Scarborough 2 S...,0 Malv...,43.806686,-79.194353
4,0 Scarborough 1 Scarborough 2 S...,0 Malv...,43.806686,-79.194353


#### Setting Radius and Limit for venue search

In [121]:
radius=500
LIMIT=100

In [125]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [123]:

Tor_venues = getNearbyVenues(names=Tor_df['Neighbourhood'],
                                   latitudes=Tor_df['Latitude'],
                                   longitudes=Tor_df['Longitude']
                                  )

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale, Willowdale East
York Mills West
Willowdale, Willowdale West
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto, Broadview North (Old East York)
The Danforth West, 

In [127]:
Tor_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Chris Effects Painting,43.784343,-79.163742,Construction & Landscaping
2,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Great Shine Window Cleaning,43.783145,-79.157431,Home Service
3,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank


#### See how many venues we have in each neighborhood

In [131]:
Tor_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Agincourt,4,4,4,4,4,4
"Alderwood, Long Branch",8,8,8,8,8,8
"Bathurst Manor, Wilson Heights, Downsview North",21,21,21,21,21,21
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",25,25,25,25,25,25
...,...,...,...,...,...,...
"Willowdale, Willowdale East",34,34,34,34,34,34
"Willowdale, Willowdale West",5,5,5,5,5,5
Woburn,3,3,3,3,3,3


In [133]:
# one hot encoding
Tor_onehot = pd.get_dummies(Tor_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Tor_onehot['Neighbourhood'] = Tor_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [Tor_onehot.columns[-1]] + list(Tor_onehot.columns[:-1])
Tor_onehot = Tor_onehot[fixed_columns]

Tor_onehot.head()

,Neighbourhood,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
